In [9]:
import codecs
import re
import nltk
import collections
import numpy
import subprocess
import json
import multiprocessing
import os
import gc
import sklearn
import networkx as nx

In [10]:
letters_regex = re.compile(r'[а-яА-ЯёЁ]+')
def has_letter(s):
    return letters_regex.match(s)

In [11]:
def mystem_analyze(text):
    p = subprocess.Popen(["mystem", "--format=json", "-nigfcsd"],
                         stdin=subprocess.PIPE,
                         stdout=subprocess.PIPE,
                         stderr=subprocess.PIPE)
    stdout, stderr = p.communicate(text.encode('utf-8'))
    res_str = stdout.decode('utf-8')
    return json.loads('[%s]' % (",".join(res_str.splitlines())))


In [12]:
def split_by_sentence_marker(analysis):
    last = 0
    for i, a in enumerate(analysis):
        if a['text'] == '\\s':
            yield analysis[last:i]
            last = i + 1
    if analysis[last:]:
        yield analysis[last:]


In [13]:
TextFeatures = collections.namedtuple('TextFeatures', 'avg_words_in_sentence avg_word_length rel_voc_size rel_hapax_legomena_count rel_pos_count rel_char_count rel_common_word_count')

def char_range(start, end):
    return [chr(char) for char in range(ord(start), ord(end) + 1)]

mystem_pos = ['A', 'ADV', 'ADVPRO', 'ANUM', 'APRO', 'COM', 'CONJ', 'INTJ', 'NUM', 'PART', 'PR', 'S', 'SPRO', 'V']
alpha_chars = char_range('а', 'я') + char_range('А', 'Я') + ['ё', 'Ё']
num_chars = char_range('0', '9')
punct_chars = list(',.?!-:;"\'')
most_freq_words = ['и', 'в', 'не', 'он', 'на', 'я', 'что', 'тот', 'быть', 'с', 'а', 'весь', 'это', 'как', 'она', 'по', 'но', 'они', 'к', 'у', 'ты', 'из', 'мы', 'за', 'вы', 'так', 'же', 'от', 'сказать', 'этот', 'который', 'мочь', 'человек', 'о', 'один', 'еще', 'бы', 'такой', 'только', 'себя', 'свое', 'какой', 'когда', 'уже', 'для', 'вот', 'кто', 'да', 'говорить', 'год', 'знать', 'мой', 'до', 'или', 'если', 'время', 'рука', 'нет', 'самый', 'ни', 'стать', 'большой', 'даже', 'другой', 'наш', 'свой', 'ну', 'под', 'где', 'дело', 'есть', 'сам', 'раз', 'чтобы', 'два', 'там', 'чем', 'глаз', 'жизнь', 'первый', 'день', 'тута', 'во', 'ничто', 'потом', 'очень', 'со', 'хотеть', 'ли', 'при', 'голова', 'надо', 'без', 'видеть', 'идти', 'теперь', 'тоже', 'стоять', 'друг', 'дом']

def get_features(analysis):
    words = [w for w in analysis if w.get('analysis')]
    text = "".join([w['text'] for w in analysis])
    sents = list(split_by_sentence_marker(analysis))
    words_of_sents = [[w for w in s if w.get('analysis')] for s in sents]
    lemma_count = collections.Counter([w['analysis'][0]['lex'] for w in words])
    char_count = collections.Counter(text)
    
    avg_words_in_sentence = numpy.average([len(sent) for sent in words_of_sents])
    word_count = len(words)
    avg_word_length = numpy.average([len(w['text']) for w in words])
    rel_voc_size = len(lemma_count) / word_count
    rel_hapax_legomena_count = len([w for w in lemma_count if lemma_count[w] == 1]) / word_count
    pos_count = collections.Counter(re.split('[,=()]', w['analysis'][0]['gr'])[0] for w in words)
    rel_pos_count = [pos_count[p] / word_count for p in mystem_pos]
    rel_char_count = [char_count[c] / len(text) for c in alpha_chars + num_chars + punct_chars]
    rel_common_word_count = [lemma_count[w] / word_count for w in most_freq_words]
    return TextFeatures(
        avg_words_in_sentence=avg_words_in_sentence,
        avg_word_length=avg_word_length,
        rel_voc_size=rel_voc_size,
        rel_hapax_legomena_count=rel_hapax_legomena_count,
        rel_pos_count=rel_pos_count,
        rel_char_count=rel_char_count,
        rel_common_word_count=rel_common_word_count
    )
    

In [ ]:
def get_vector(f):
    return numpy.array([f.avg_words_in_sentence, f.avg_word_length, f.rel_voc_size, f.rel_hapax_legomena_count] +
        f.rel_pos_count + f.rel_char_count + f.rel_common_word_count)

In [ ]:
def get_features_from_file(file):
    print(file)
    with codecs.open(file, encoding='utf-8') as f:
        content = f.read()
        analysis = [json.loads(l) for l in content.splitlines()]
        return get_features(analysis)

features = [get_features_from_file(os.path.join("json", file)) for file in os.listdir("json")]

json\Abramov_Pryasliny_1_Bratya-i-sestry.edbdyw.158030.fb2.txt.json
json\Abramov_Pryasliny_2_Dve-zimy-i-tri-leta.hU9IrQ.158033.fb2.txt.json
json\Abramov_Pryasliny_3_Puti-pereputya.ktaB0Q.158036.fb2.txt.json
json\Abramov_Pryasliny_4_Dom.FWiheg.158032.fb2.txt.json
json\Aksenov_Zvezdnyy-bilet.HMAp3Q.677.fb2.txt.json
json\Akunin_Priklyucheniya-Erasta-Fandorina_2_Tureckiy-gambit.xK5KVg.305713.fb2.txt.json
json\Akunin_Priklyucheniya-Erasta-Fandorina_7_Statskiy-sovetnik.5irVkw.461230.fb2.txt.json
json\Akunin_Priklyucheniya-Nikolasa-Fandorina_1_Altyn-Tolobas.Uz6tpg.131819.fb2.txt.json
json\Aleksievich_Golosa-Utopii_1_U-voyny-ne-zhenskoe-lico-.VNQLMg.70938.fb2.txt.json
json\Aleksievich_Golosa-Utopii_3_Cinkovye-malchiki.NVa1uw.427115.fb2.txt.json
json\Aleksievich_Golosa-Utopii_5_Vremya-sekond-hend.9FJziw.426019.fb2.txt.json
json\Aleksin_Anatoliy-Aleksin-Sobranie-sochineniy-v-treh-tomah_1_Bezumnaya-Evdokiya.gkQ47w.347944.fb2.txt.json
json\Aleksin_Moy-brat-igraet-na-klarnete.oGHedw.1061.fb2.txt.js

In [ ]:
book_files = os.listdir("json")
book_files

In [ ]:
features[0]._asdict().keys()

In [ ]:
feature_matrix = numpy.array([get_vector(f) for f in features])
scaled_feature_matrix = sklearn.preprocessing.scale(feature_matrix)

In [ ]:
numpy.shape(scaled_feature_matrix)

In [ ]:
eucl_dists = sklearn.metrics.pairwise.pairwise_distances(scaled_feature_matrix)
numpy.shape(eucl_dists)

In [ ]:
def dist_graph(dists, names, threshold):
    
    def author(book):
        return book.split('_')[0]
    
    g = nx.Graph()
    g.add_nodes_from((i, dict(author=author(book))) for i, book in enumerate(names))
    g.add_weighted_edges_from((i, j, 1/dist) for (i, j), dist in numpy.ndenumerate(dists) if dist <= threshold and dist > 0)
    return g

nx.write_gml(dist_graph(eucl_dists, book_files, 15), 'graph.gml')

In [ ]:
book_files[120]

In [ ]:
list(numpy.ndenumerate(eucl_dists))

In [ ]:
list(enumerate(book_files))

In [ ]:
eucl_dists[100][116]